# Process Approved Variants

This notebook:
1. Loads approved variants from the CSV file
2. Copies variants to S3 with new variant IDs (original ID * 100000)
3. Updates the Redshift database
4. Updates the API image list

In [1]:
# Import the module with all the functions
from process_approved_variants import *

In [3]:
import pandas as pd

# Attempt to read the CSV file with 'ISO-8859-1' encoding to handle potential encoding issues
df = pd.read_csv('12.csv')
# Drop any columns with 'Unnamed' in their name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# Replace string '1' with 'approved'
df['review_result'] = df['review_result'].replace('1', 'approved')
# Filter out rejected images
df_rejected = df[df['review_result'] == 'rejected']
df['id'] = df['id'].astype(int)
df = df[df['category_name'] == "Restaurants & Bars"]
df

,id,email_subject,category_name,vertical,sub_category_name,revenue_last_7_days,revenue_rank,image_id_pos_0,image_url_pos_0,extension,status,s3_url,token_info,processed_timestamp,prompt,prompt_source,error,review_result,review_notes
9,41523285,Mahony's Steakhouse Dining,Restaurants & Bars,LOCAL,Steaks,243.80,10,1723813,https://static.wowcher.co.uk/images/deal/41523...,jpg,success,https://static.wowcher.co.uk/images/deal/41523...,"{'Total tokens': '8641', 'Input tokens': '2433...",2025-08-11 10:15:32.538972,<imagePrompt>\n <title>High-Resolution Promot...,restaurants_&_bars,NaN,approved,NaN
22,41539006,Steak or 2-Course Meal for Two,Restaurants & Bars,LOCAL,Steaks,98.82,23,1726797,https://static.wowcher.co.uk/images/deal/41539...,jpg,success,https://static.wowcher.co.uk/images/deal/41539...,"{'Total tokens': '8644', 'Input tokens': '2436...",2025-08-11 10:15:34.855990,<imagePrompt>\n <title>High-Resolution Promot...,restaurants_&_bars,NaN,approved,NaN
32,38870054,An Evening with Whitney,Restaurants & Bars,LOCAL,Cabaret,56.17,33,1727533,https://static.wowcher.co.uk/images/deal/38870...,jpg,success,https://static.wowcher.co.uk/images/deal/38870...,"{'Total tokens': '7243', 'Input tokens': '1035...",2025-08-11 10:15:36.705574,<imagePrompt>\n <title>High-Resolution Promot...,restaurants_&_bars,NaN,rejected,NaN
35,41585591,Animal Kingdom Afternoon Tea,Restaurants & Bars,LOCAL,Afternoon Tea,48.00,36,1732182,https://static.wowcher.co.uk/images/deal/41585...,jpg,success,https://static.wowcher.co.uk/images/deal/41585...,"{'Total tokens': '8639', 'Input tokens': '2431...",2025-08-11 10:15:37.243131,<imagePrompt>\n <title>High-Resolution Promot...,restaurants_&_bars,NaN,rejected,NaN
36,41650053,An Evening of Laughter with Stan Boardman,Restaurants & Bars,LOCAL,Cabaret,31.85,37,1733459,https://static.wowcher.co.uk/images/deal/41650...,jpg,success,https://static.wowcher.co.uk/images/deal/41650...,"{'Total tokens': '7248', 'Input tokens': '1040...",2025-08-11 10:15:37.398381,<imagePrompt>\n <title>High-Resolution Promot...,restaurants_&_bars,NaN,rejected,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,41585600,2-Course Sunday Roast with Wine - Leicester,Restaurants & Bars,LOCAL,British,0.00,64,1726894,https://static.wowcher.co.uk/images/deal/41585...,jpg,success,https://static.wowcher.co.uk/images/deal/41585...,"{'Total tokens': '8645', 'Input tokens': '2437...",2025-08-11 10:26:09.423043,<imagePrompt>\n <title>High-Resolution Promot...,restaurants_&_bars,NaN,approved,NaN
356,39908211,Michael Jackson Tribute Show,Restaurants & Bars,LOCAL,Cabaret,0.00,64,1586236,https://static.wowcher.co.uk/images/deal/39908...,jpg,success,https://static.wowcher.co.uk/images/deal/39908...,"{'Total tokens': '7243', 'Input tokens': '1035...",2025-08-11 10:26:09.767397,<imagePrompt>\n <title>High-Resolution Promot...,restaurants_&_bars,NaN,rejected,NaN
359,28969917,Bottomless Brunch for 2,Restaurants & Bars,LOCAL,British,0.00,64,1152852,https://static.wowcher.co.uk/images/deal/28969...,jpg,success,https://static.wowcher.co.uk/images/deal/28969...,"{'Total tokens': '8642', 'Input tokens': '2434...",2025-08-11 10:26:10.335203,<imagePrompt>\n <title>High-Resolution Promot...,restaurants_&_bars,NaN,rejected,NaN
361,33794153,On The Rocks Bar Crawl Brighton,Restaurants & Bars,LOCAL,British,0.00,64,1383299,https://static.wowcher.co.uk/images/deal/33794...,jpg,success,https://static.wowcher.co.uk/images/deal/33794...,"{'Total tokens': '8292', 'Input tokens': '2084...",2025-08-11 10:26:10.666933,<imagePrompt>\n <title>High-Resolution Promot...,restaurants_&_bars,NaN,approved,NaN


## 1. Load and filter approved images

In [4]:
import pandas as pd
import json
import re

# Function to safely parse the token info column
def extract_cost(token_info):
    try:
        # Handle if it's already a dict
        if isinstance(token_info, dict):
            return float(token_info.get('Cost', '0').replace('$', ''))
        
        # Try to parse as JSON if it's a string
        if isinstance(token_info, str):
            # Clean up any potential invalid JSON formatting
            cleaned = token_info.replace("'", '"')
            data = json.loads(cleaned)
            return float(data.get('Cost', '0').replace('$', ''))
    except:
        # If JSON parsing fails, try regex
        try:
            cost_match = re.search(r"'Cost': '\$([\d.]+)'", str(token_info))
            if cost_match:
                return float(cost_match.group(1))
        except:
            pass
    return 0.0

# Extract cost from token info column
df['extracted_cost'] = df['token_info'].apply(extract_cost)

# Calculate total cost and count by review result
cost_summary = df.groupby('review_result').agg(
    total_cost=('extracted_cost', 'sum'),
    count=('extracted_cost', 'count')
).reset_index()

# Format the cost with $ sign
cost_summary['total_cost'] = cost_summary['total_cost'].apply(lambda x: f"${x:.6f}")

print("Cost summary by approval status:")
print(cost_summary)

# Calculate overall total
overall_total_cost = df['extracted_cost'].sum()
overall_count = len(df)

print(f"\nOverall total cost: ${overall_total_cost:.6f}")
print(f"Overall count: {overall_count}")

Cost summary by approval status:
  review_result  total_cost  count
0      approved  $33.868505    129
1      rejected   $6.546160     25

Overall total cost: $40.414665
Overall count: 154


## 2. Prepare data for Redshift

In [5]:
# Prepare the data for Redshift
approved_df = df[(df['review_result'] == 'approved') & (df['image_id_pos_0'].notna()) & (df['image_id_pos_0'].apply(lambda x: isinstance(x, (int, float))))]
approved_df['image_id_pos_0'] = approved_df['image_id_pos_0'].apply(lambda x: int(x) if isinstance(x, float) else x)
redshift_df = prepare_for_redshift(approved_df)
# Display the first few rows of the prepared data
redshift_df.head()

Preparing data structure for Redshift...
Prepared 129 rows for processing


/var/folders/dk/zpttw0wn0192zryb4bdz19lr0000gn/T/ipykernel_85984/1705187957.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  approved_df['image_id_pos_0'] = approved_df['image_id_pos_0'].apply(lambda x: int(x) if isinstance(x, float) else x)


,deal_voucher_id,claid_prompt,status,original_image_id,variant_image_id,batch_name,enter_test_ts,exit_test_ts,list_name,s3_url,original_url,processed_status,final_s3_url,new_oracle_id
9,41523285,,1,1723813,None,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/41523...,https://static.wowcher.co.uk/images/deal/41523...,False,None,None
22,41539006,,1,1726797,None,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/41539...,https://static.wowcher.co.uk/images/deal/41539...,False,None,None
39,40031285,,1,1594012,None,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/40031...,https://static.wowcher.co.uk/images/deal/40031...,False,None,None
41,27998237,,1,1091341,None,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/27998...,https://static.wowcher.co.uk/images/deal/27998...,False,None,None
42,39631797,,1,1602307,None,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/39631...,https://static.wowcher.co.uk/images/deal/39631...,False,None,None


In [8]:
# Connect to Oracle and execute the SQL query
import oracledb

import os
from dotenv import load_dotenv

load_dotenv()

def get_oracle_connection():
    connection = oracledb.connect(
        user=os.getenv("ORACLE_USER"), 
        password=os.getenv("ORACLE_PASSWORD"), 
        dsn=os.getenv("ORACLE_DSN")
    )
    return connection

# Fetch data from Oracle
def fetch_data_from_oracle(deal_voucher_ids):
    query = """
    SELECT deal_voucher_id, id AS original_image_id
    FROM deal_voucher_image
    WHERE deal_voucher_id IN ({})
    AND position = 0
    """.format(','.join(map(str, deal_voucher_ids)))

    connection = get_oracle_connection()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    connection.close()
    return result

# Update redshift_df with data from Oracle
deal_voucher_ids = redshift_df['deal_voucher_id'].to_list()
oracle_data = fetch_data_from_oracle(deal_voucher_ids)

# Convert the fetched data to a DataFrame
oracle_df = pd.DataFrame(oracle_data, columns=['deal_voucher_id', 'original_image_id'])

# Merge the Oracle data with redshift_df
redshift_df = redshift_df.merge(oracle_df, on='deal_voucher_id', how='left', suffixes=('', '_oracle'))

# Update the original_image_id in redshift_df with the one from Oracle if available
redshift_df['original_image_id'] = redshift_df['original_image_id_oracle'].combine_first(redshift_df['original_image_id'])

# Drop the temporary column
redshift_df.drop(columns=['original_image_id_oracle'], inplace=True)


## 3. Copy variants to S3 with new variant IDs

In [9]:
display(redshift_df)
processed_df = process_approved_variants_with_oracle(redshift_df)
# Display a summary of the processed data
success_count = processed_df['processed_status'].sum()
print(f"Successfully processed {success_count} of {len(processed_df)} variants")
redshift_df.to_csv('redshift_df.csv', index=False)

,deal_voucher_id,claid_prompt,status,original_image_id,variant_image_id,batch_name,enter_test_ts,exit_test_ts,list_name,s3_url,original_url,processed_status,final_s3_url,new_oracle_id
0,41523285,,1,1737811,173781100000,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/41523...,https://static.wowcher.co.uk/images/deal/41523...,True,https://static.wowcher.co.uk/images/deal/41523...,1737811
1,41539006,,1,1737809,173780900000,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/41539...,https://static.wowcher.co.uk/images/deal/41539...,True,https://static.wowcher.co.uk/images/deal/41539...,1737809
2,40031285,,1,1737799,173779900000,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/40031...,https://static.wowcher.co.uk/images/deal/40031...,True,https://static.wowcher.co.uk/images/deal/40031...,1737799
3,27998237,,1,1737796,173779600000,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/27998...,https://static.wowcher.co.uk/images/deal/27998...,True,https://static.wowcher.co.uk/images/deal/27998...,1737796
4,39631797,,1,1737803,173780300000,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/39631...,https://static.wowcher.co.uk/images/deal/39631...,True,https://static.wowcher.co.uk/images/deal/39631...,1737803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,28272044,,1,1737912,173791200000,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/28272...,https://static.wowcher.co.uk/images/deal/28272...,True,https://static.wowcher.co.uk/images/deal/28272...,1737912
125,41539488,,1,1737913,173791300000,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/41539...,https://static.wowcher.co.uk/images/deal/41539...,True,https://static.wowcher.co.uk/images/deal/41539...,1737913
126,41585600,,1,1737914,173791400000,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/41585...,https://static.wowcher.co.uk/images/deal/41585...,True,https://static.wowcher.co.uk/images/deal/41585...,1737914
127,33794153,,1,1737915,173791500000,OPEN AI Images,2025-08-11 15:17:23,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/33794...,https://static.wowcher.co.uk/images/deal/33794...,True,https://static.wowcher.co.uk/images/deal/33794...,1737915


Processing Approved Variants with Oracle: 100%|██████████| 129/129 [00:16<00:00,  7.82it/s]

Successfully processed 129 of 129 variants
Saved processed data to processed_approved_variants.csv
Successfully processed 129 of 129 variants


## 4. Upload prepared data to S3 for Redshift

In [10]:
# Upload the prepared data to S3 for Redshift
s3_url = upload_to_s3_for_redshift(processed_df)

if s3_url:
    print(f"Successfully uploaded data to S3: {s3_url}")
else:
    print("Failed to upload data to S3")

Preparing data for Redshift...
Found 129 successfully processed variants for Redshift
Saved Redshift data to redshift_upload_data.csv
Uploaded data to S3: https://static.wowcher.co.uk/temp/ai_image_variants_20250811152044.csv
Successfully uploaded data to S3: https://static.wowcher.co.uk/temp/ai_image_variants_20250811152044.csv


## 5. Copy data from S3 to Redshift

In [11]:
# Only proceed if we have a valid S3 URL
if s3_url:
    # Copy the data from S3 to Redshift
    success = copy_s3_to_redshift(s3_url)
    if success:
        print("Successfully copied data to Redshift")
    else:
        print("Failed to copy data to Redshift")
else:
    print("Skipping Redshift update due to missing S3 URL")
    



Copying data to Redshift...
Successfully copied 7983 rows to Redshift table
Successfully copied data to Redshift


## 6. Update the image list in the API

In [12]:
# Only proceed if we successfully copied to Redshift
if 'success' in locals() and success:
    # Update the image list in the API
    result = update_image_list()
    if result:
        print("Successfully updated the image list in the API")
    else:
        print("Failed to update the image list in the API")
else:
    print("Skipping API update due to failed Redshift update")

Updating image list in API...
Found 2918 unique NEW variant images to add to list
Successfully updated test list with 2918 NEW variant images
Successfully updated the image list in the API


# Purge Cache on image urls. Usefull if making images where a variant already existed and now need it to show the new variant


In [13]:
ids_str = df['id'].apply(lambda x: str(int(x))).to_list()
print('re-index in chunks of 50 at this url: https://search-ingest-gr.wowcher.co.uk/')
# Iterate over the list in chunks of 50
for i in range(0, len(ids_str), 250):
    # Join the chunk into a comma-separated string and print
    print(','.join(ids_str[i:i+250]))


re-index in chunks of 50 at this url: https://search-ingest-gr.wowcher.co.uk/
41523285,41539006,38870054,41585591,41650053,41539133,40031285,27998237,39631797,39853514,40436576,40881805,40271979,41539314,28506528,41635200,39779927,39263141,41618877,26841070,29245322,33931249,37996565,37475856,39550836,39182156,31039061,39832060,25649505,40511602,41153888,24622654,41288788,41585593,38254651,36807651,38583445,27271061,27271034,40601562,41271080,41523255,32322230,30458615,30922273,29400665,37359641,15262454,41061603,28956112,41635331,30074350,37550661,39695299,40350890,40511600,41306327,34903987,39182152,38191203,32598389,38239286,38583412,41139782,41474436,32322614,29430661,27289709,29950213,32017253,38239765,26530173,40446201,34772789,32515564,32459634,40282124,40325136,38600529,38240757,22646686,40973998,27998107,39868511,40499701,40765336,39966745,32322215,41306842,28883273,32383931,26875829,40974015,30990108,39483948,34772801,26994515,27523744,28287781,40272845,41523727,36957809,3983

In [11]:
import psycopg2

conn_params = {
    "host": "bi-redshift.intwowcher.co.uk",
    "port": 5439,
    "dbname": "wowdwhprod",
    "user": "jenkins",
    "password": "9SDy1ffdfTV7"
}

def cleanup_images():
    sequence_query = """
    UPDATE temp.opt_image_variants t1
    SET 
        status = 5,
        exit_test_ts = t2.enter_test_ts
    FROM temp.opt_image_variants t2
    WHERE (
        (t1.original_image_id = t2.original_image_id AND t1.id < t2.id)
        OR
        (t1.deal_voucher_id = t2.deal_voucher_id AND t1.original_image_id != t2.original_image_id AND t1.batch_name= 'OPEN AI Images' AND t2.batch_name= 'OPEN AI Images')
    )
    AND t1.status = 1 
    AND t2.status = 1;
    """
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(sequence_query)
            row_count = cur.rowcount
            conn.commit()
            print(f"Successfully updated {row_count} earlier variants with exit_test_ts and status=5")
            print("Latest variants for each deal remain unchanged with status=1")

# Execute the bulk update
sequence_results = cleanup_images()

Successfully updated 0 earlier variants with exit_test_ts and status=5
Latest variants for each deal remain unchanged with status=1


In [12]:
import psycopg2
import pandas as pd

sql = """
select 
'https://static.wowcher.co.uk/images/deal/'||oiv.deal_voucher_id||'/'||oiv.original_image_id||'00000.'||dvi.extension as url
from temp.opt_image_variants oiv
join real.deal_voucher_image dvi on dvi.id = oiv.original_image_id
where oiv.status = 1
and trunc(enter_test_ts) = trunc(sysdate)
"""

REDSHIFT_CONFIG = {
    'host': 'bi-redshift.intwowcher.co.uk',
    'port': 5439,
    'dbname': 'wowdwhprod',
    'user': 'jenkins',
    'password': '9SDy1ffdfTV7'
}

conn = psycopg2.connect(**REDSHIFT_CONFIG)

# Use pandas to read SQL query
urls =  pd.read_sql_query(sql, conn)['url'].to_list()

# Close connection
conn.close()

/var/folders/dk/zpttw0wn0192zryb4bdz19lr0000gn/T/ipykernel_71610/287702055.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  urls =  pd.read_sql_query(sql, conn)['url'].to_list()


In [13]:
import requests 
import json

def purge_cache(image_urls):
    chunk_size = 30 # cloudflare purge api takes max 30 urls per request
    chunked_list = [image_urls[i:i+chunk_size] for i in range(0, len(image_urls), chunk_size)]
    api_url = "https://api.cloudflare.com/client/v4/zones/4fec7e02d5c45deb9f67452873708896/purge_cache"
    api_key = "Bearer IT-Lr8A8LOipKdcGKkJU8Q5ciM8jfy3KbtfWkDnK"
    headers = {"Authorization": api_key,
               "Content-Type":"application/json"
               }
    for chunk in chunked_list:
        payload = {"files": []}
        #print("Clearing cache of: ", chunk)
        payload['files'] = chunk
        response = requests.post(url=api_url, data=json.dumps(payload), headers=headers)
        print(response.json())
purge_cache(urls)